In [3]:
import re
from math import gcd

# Function to compute the extended gcd (Extended Euclidean Algorithm)
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0
    g, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return g, x, y

# Function to parse the input file
def parse_input(file_path):
    with open(file_path, 'r') as f:
        data = f.read().strip().split('\n\n')
    machines = []
    for machine in data:
        lines = machine.split('\n')
        # Parse button A and B movements
        A_match = re.search(r"X\+(\d+), Y\+(\d+)", lines[0])
        B_match = re.search(r"X\+(\d+), Y\+(\d+)", lines[1])
        P_match = re.search(r"X=(\d+), Y=(\d+)", lines[2])

        machines.append({
            "A_x": int(A_match.group(1)), "A_y": int(A_match.group(2)),
            "B_x": int(B_match.group(1)), "B_y": int(B_match.group(2)),
            "P_x": int(P_match.group(1)), "P_y": int(P_match.group(2))
        })
    return machines

# Function to solve a single claw machine using the updated approach
def solve_machine_extended(machine, offset=0):
    A_x, A_y = machine["A_x"], machine["A_y"]
    B_x, B_y = machine["B_x"], machine["B_y"]
    P_x, P_y = machine["P_x"] + offset, machine["P_y"] + offset

    # Solve for x_A and x_B in the x-coordinate equation
    g_x, x_x, y_x = extended_gcd(A_x, B_x)
    if P_x % g_x != 0:
        return None  # No solution for x

    # Solve for x_A and x_B in the y-coordinate equation
    g_y, x_y, y_y = extended_gcd(A_y, B_y)
    if P_y % g_y != 0:
        return None  # No solution for y

    # Scale solutions for P_x and P_y
    k_x = P_x // g_x
    x_x *= k_x
    y_x *= k_x

    k_y = P_y // g_y
    x_y *= k_y
    y_y *= k_y

    # General solution forms
    A_x_step, B_x_step = B_x // g_x, -A_x // g_x
    A_y_step, B_y_step = B_y // g_y, -A_y // g_y

    # Adjust solutions to ensure non-negative x_A and x_B
    def adjust_to_non_negative(x, y, step_x, step_y):
        t_min = max(-x // step_x if step_x != 0 else 0,
                    -y // step_y if step_y != 0 else 0)
        x += t_min * step_x
        y += t_min * step_y
        return x, y

    x_x, y_x = adjust_to_non_negative(x_x, y_x, A_x_step, B_x_step)
    x_y, y_y = adjust_to_non_negative(x_y, y_y, A_y_step, B_y_step)

    # Check if solutions are valid
    if x_x < 0 or y_x < 0 or x_y < 0 or y_y < 0:
        return None

    # Compute cost
    cost_x = 3 * x_x + 1 * y_x
    cost_y = 3 * x_y + 1 * y_y

    # Return minimum cost if both solutions are valid
    return min(cost_x, cost_y)

# Solve all claw machines with the updated approach
def solve_all_machines_extended(file_path, offset=0):
    machines = parse_input(file_path)
    results = []
    for machine in machines:
        min_cost = solve_machine_extended(machine, offset)
        results.append(min_cost)
    num_prizes_won = sum(1 for cost in results if cost is not None)
    total_min_cost = sum(cost for cost in results if cost is not None)
    return results, num_prizes_won, total_min_cost

# File path
file_path = 'input.txt'

# Solve Part One
part_one_results, part_one_prizes, part_one_cost = solve_all_machines_extended(file_path)

# Solve Part Two with offset of 10^12
part_two_results, part_two_prizes, part_two_cost = solve_all_machines_extended(file_path, offset=10**12)

# Print results
print("Part One:")
print(f"Prizes won: {part_one_prizes}, Total cost: {part_one_cost}")

print("Part Two:")
print(f"Prizes won: {part_two_prizes}, Total cost: {part_two_cost}")

Part One:
Prizes won: 265, Total cost: 78862
Part Two:
Prizes won: 248, Total cost: 13711607197244


In [2]:
import re

# Set part: 1 for first part, 2 for the adjusted coordinates (second part)
part = 2

# Read the input from 'input.txt'
with open("input.txt", "r") as f:
    inp = f.read()

result = 0
pattern = re.compile(r"(\d+)")
machines = pattern.findall(inp)

for i in range(len(machines) // 6):
    (ax, ay, bx, by, tx, ty) = [int(machines[6*i+j]) for j in range(6)]
    # Adjust the prize location based on part
    tx += (part - 1) * 10000000000000
    ty += (part - 1) * 10000000000000

    # Solve the linear equations:
    # ax*a + bx*b = tx
    # ay*a + by*b = ty

    # Using the derived formula from your code:
    # a = ( (by*tx) - (bx*ty) ) / ( (by*ax) - (bx*ay) )
    denominator = (by * ax - bx * ay)
    if denominator == 0:
        # If denominator is 0, no solution for a or b in simple form
        continue

    a = ((by * tx) - (bx * ty)) / denominator
    # After finding a, solve for b
    if bx != 0:
        b = (tx - ax * a) / bx
    else:
        # If bx=0, then b cannot be computed this way; skip if needed
        continue

    # Check if a and b are integers and non-negative
    if a.is_integer() and b.is_integer() and a >= 0 and b >= 0:
        a = int(a)
        b = int(b)
        # Cost is 3 tokens per A press and 1 token per B press
        tokens = a * 3 + b
        result += tokens

# Display the result
result

83232379451012